<a href="https://colab.research.google.com/github/mounirhomrani2-prog/tts-bambara-project/blob/training-result/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Installation Libraries**

In [5]:
!pip install --upgrade pip

In [ ]:
!apt-get update -qq 2>/dev/null || true
!apt-get install -y build-essential espeak espeak-data libespeak1 libespeak-dev
print("📦 Installation de CoquiTTS...")
!pip install coqui-tts
!apt-get update -qq 2>/dev/null || true
!apt-get install -y build-essential espeak espeak-data libespeak1 libespeak-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
The following additional packages will be installed:
  libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak-dev libespeak1 libportaudio2 libsonic0
0 upgraded, 6 newly installed, 0 to remove and 93 not upgraded.
Need to get 1,575 kB of archives.
After this operation, 3,802 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak-data amd64 1.48.15+dfsg-3 [1,085 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libespeak1 amd64 1.48.15+dfsg-3 [156 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak amd64 1.4

In [ ]:
!pip install torch torchaudio
!pip install numpy scipy librosa soundfile

In [23]:
# Installation de Coqui TTS et dépendances
!pip install -q pandas pyarrow soundfile librosa

# Vérifier l'installation
import TTS
print(f"✅ TTS version : {TTS.__version__}")

# Imports nécessaires
import os
import json
import soundfile as sf
from pathlib import Path
import shutil

✅ TTS version : 0.27.3


# **Verification of the installed environment**

In [6]:
# Re-vérification après installation
print("🔧 Re-vérification après installation...")

try:
    import TTS
    print(f"✅ CoquiTTS: {TTS.__version__}")

    # Test d'import des modules principaux
    from TTS.tts.configs.shared_configs import BaseDatasetConfig
    from TTS.tts.configs.vits_config import VitsConfig
    print("✅ Modules TTS importés avec succès")

except Exception as e:
    print(f"❌ Problème avec CoquiTTS: {e}")

# Vérification de PyTorch (important pour GPU)
import torch
print(f"🔥 PyTorch: {torch.__version__}")
print(f"🔥 CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🔥 GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️  Mode CPU seulement")

print("\n" + "="*50)

🔧 Re-vérification après installation...
✅ CoquiTTS: 0.27.3
✅ Modules TTS importés avec succès
🔥 PyTorch: 2.9.0+cu126
🔥 CUDA disponible: True
🔥 GPU: NVIDIA A100-SXM4-40GB



# **Defines name, formatter and path of the dataset**

In [7]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)


# **Copy dataset from Drive**

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from google.colab import drive
import shutil
import os

# Monter Google Drive (déjà monté)
# drive.mount('/content/drive')

print("=" * 50)
print("EXPLORATION DE GOOGLE DRIVE")
print("=" * 50)

# Lister le contenu principal
print("\n📁 Contenu de MyDrive :")
!ls -lh "/content/drive/MyDrive/Bambara_TTS/afvoices_permanent/human-corrected"

EXPLORATION DE GOOGLE DRIVE

📁 Contenu de MyDrive :
total 16G
-rw------- 1 root root 366M Jan 15 09:51 test-00000-of-00004.parquet
-rw------- 1 root root 357M Jan 15 09:51 test-00001-of-00004.parquet
-rw------- 1 root root    0 Jan 15 10:50 test-00002-of-00004.parquet
-rw------- 1 root root 357M Jan 15 09:51 test-00003-of-00004.parquet
-rw------- 1 root root 430M Jan 15 09:51 train-00001-of-00126.parquet
-rw------- 1 root root 428M Jan 15 09:51 train-00002-of-00126.parquet
-rw------- 1 root root 441M Jan 15 09:49 train-00003-of-00126.parquet
-rw------- 1 root root    0 Jan 15 10:50 train-00004-of-00126.parquet
-rw------- 1 root root 433M Jan 15 09:50 train-00008-of-00126.parquet
-rw------- 1 root root 431M Jan 15 09:51 train-00010-of-00126.parquet
-rw------- 1 root root 449M Jan 15 09:43 train-00014-of-00126.parquet
-rw------- 1 root root    0 Jan 15 10:50 train-00015-of-00126.parquet
-rw------- 1 root root 444M Jan 15 09:47 train-00017-of-00126.parquet
-rw------- 1 root root 432M Jan 

In [11]:
import shutil
import os

print("=" * 50)
print("COPIE DES DONNÉES DEPUIS GOOGLE DRIVE")
print("=" * 50)

# Chemin source sur Drive
source_path = "/content/drive/MyDrive/Bambara_TTS/afvoices_permanent"

# Chemin de destination dans Colab
destination_path = "/content/afvoices"

# Vérifier si la source existe
if os.path.exists(source_path):
    print(f"✅ Source trouvée : {source_path}")

    # Supprimer la destination si elle existe déjà
    if os.path.exists(destination_path):
        print(f"🗑️ Suppression de l'ancien dossier...")
        shutil.rmtree(destination_path)

    # Copier les données
    print(f"📋 Copie en cours...")
    shutil.copytree(source_path, destination_path)

    print(f"✅ Données copiées avec succès vers : {destination_path}")

    # Vérifier le contenu
    print("\n📁 Contenu copié :")
    !ls -lh /content/afvoices/

else:
    print(f"❌ Source introuvable : {source_path}")
    print("\n📁 Vérifiez le chemin. Contenu de Bambara_TTS :")
    !ls -lh "/content/drive/MyDrive/Bambara_TTS/"


COPIE DES DONNÉES DEPUIS GOOGLE DRIVE
✅ Source trouvée : /content/drive/MyDrive/Bambara_TTS/afvoices_permanent
🗑️ Suppression de l'ancien dossier...
📋 Copie en cours...
✅ Données copiées avec succès vers : /content/afvoices

📁 Contenu copié :
total 16K
drwx------ 3 root root 4.0K Jan 15 10:47 human-corrected
-rw------- 1 root root 8.6K Jan 15 09:43 README.md


# **Verification of dataset**

In [12]:
import os

# Vérifier le contenu du dossier
!ls -la afvoices/

# Explorer la structure
for root, dirs, files in os.walk('afvoices'):
    level = root.replace('afvoices', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Afficher les 5 premiers fichiers
        print(f'{subindent}{file}')
    if len(files) > 5:
        print(f'{subindent}... et {len(files)-5} autres fichiers')

total 28
drwx------ 4 root root 4096 Jan 15 10:47 .
drwxr-xr-x 1 root root 4096 Jan 18 08:22 ..
drwx------ 9 root root 4096 Jan 15 10:50 .git
-rw------- 1 root root    0 Jan 15 10:50 .gitattributes
drwx------ 3 root root 4096 Jan 15 10:47 human-corrected
-rw------- 1 root root 8806 Jan 15 09:43 README.md
afvoices/
  README.md
  .gitattributes
  .git/
    config
    HEAD
    packed-refs
    lfs/
      tmp/
      incomplete/
      objects/
        50/
          c6/
            50c63b75f4075587801b87054b4b3c2781929ad15e683caf5d5c56f90b163b55
        38/
          bf/
            38bf0fd987832cf0767d6fac8f93b70958c6f8d45007ceb84efbdf0ed529187a
        3a/
          d4/
            3ad4bc8d9f3178a774411e64c7d85b85282d75e74a528d94484adececd9053c3
        d2/
          fd/
            d2fd93f8eabaeef7c709c7163bde315efbf82b400705393f85f4242dacbfb6ee
        ea/
          84/
            ea84147317660def03421180c92167716cd61beef1c2afd4e5a90eef8b396aef
        5a/
          d5/
            5ad5e

In [13]:
import pandas as pd
import os
from pathlib import Path
import glob

# Créer les dossiers nécessaires
output_dir = "tts_dataset"
wavs_dir = os.path.join(output_dir, "wavs")
os.makedirs(wavs_dir, exist_ok=True)

print("=" * 50)
print("VÉRIFICATION DES FICHIERS PARQUET")
print("=" * 50)

# Charger tous les fichiers parquet du dossier human-corrected
hc_files = glob.glob('afvoices/human-corrected/*.parquet')
print(f"\n📊 Nombre de fichiers trouvés : {len(hc_files)}\n")

# Vérifier la taille de chaque fichier et charger uniquement les valides
all_data = []
valid_files = 0
invalid_files = 0

for file in hc_files:
    file_size = os.path.getsize(file)

    if file_size > 0:
        try:
            df = pd.read_parquet(file)
            all_data.append(df)
            valid_files += 1
            print(f"✅ {os.path.basename(file)} - {file_size} bytes - {len(df)} lignes")
        except Exception as e:
            invalid_files += 1
            print(f"❌ {os.path.basename(file)} - Erreur : {str(e)[:50]}")
    else:
        invalid_files += 1
        print(f"⚠️ {os.path.basename(file)} - Fichier vide (0 bytes)")

print("\n" + "=" * 50)
print("RÉSUMÉ")
print("=" * 50)
print(f"✅ Fichiers valides : {valid_files}")
print(f"❌ Fichiers invalides/vides : {invalid_files}")

if all_data:
    df_combined = pd.concat(all_data, ignore_index=True)
    print(f"📏 Total d'échantillons chargés : {len(df_combined)}")
    print(f"\nColonnes : {list(df_combined.columns)}")
    print("\nAperçu :")
    print(df_combined.head(3))
else:
    print("⚠️ Aucune donnée valide trouvée !")


VÉRIFICATION DES FICHIERS PARQUET

📊 Nombre de fichiers trouvés : 55

⚠️ train-00025-of-00126.parquet - Fichier vide (0 bytes)
⚠️ train-00052-of-00126.parquet - Fichier vide (0 bytes)
✅ train-00030-of-00126.parquet - 452374331 bytes - 2010 lignes
✅ test-00001-of-00004.parquet - 374157138 bytes - 1680 lignes
✅ train-00014-of-00126.parquet - 470024102 bytes - 2011 lignes
⚠️ test-00002-of-00004.parquet - Fichier vide (0 bytes)
⚠️ train-00023-of-00126.parquet - Fichier vide (0 bytes)
⚠️ train-00045-of-00126.parquet - Fichier vide (0 bytes)
✅ train-00041-of-00126.parquet - 463309516 bytes - 2010 lignes
✅ train-00051-of-00126.parquet - 448147260 bytes - 2010 lignes
✅ train-00076-of-00126.parquet - 446324155 bytes - 2010 lignes
✅ train-00063-of-00126.parquet - 452207974 bytes - 2010 lignes
✅ train-00043-of-00126.parquet - 466066321 bytes - 2010 lignes
✅ train-00055-of-00126.parquet - 455429727 bytes - 2010 lignes
✅ train-00039-of-00126.parquet - 453457519 bytes - 2010 lignes
⚠️ train-00020-of

In [15]:
import os
import glob

print("=" * 50)
print("SUPPRESSION DES FICHIERS VIDES")
print("=" * 50)

hc_files = glob.glob('afvoices/human-corrected/*.parquet')
deleted_count = 0

for file in hc_files:
    file_size = os.path.getsize(file)
    if file_size == 0:
        os.remove(file)
        deleted_count += 1
        print(f"🗑️ Supprimé : {os.path.basename(file)}")

print(f"\n✅ {deleted_count} fichiers vides supprimés")
print(f"📊 Fichiers restants : {len(glob.glob('afvoices/human-corrected/*.parquet'))}")


SUPPRESSION DES FICHIERS VIDES

✅ 0 fichiers vides supprimés
📊 Fichiers restants : 38


## **Save audio files from DataFrame**

This step extracts the audio byte data from the `df_combined` DataFrame and saves each audio segment as a `.wav` file in the `tts_dataset/wavs` directory. This is crucial as Coqui TTS training requires individual audio files on disk.

We ensure the filenames match the format expected by the metadata file (`audio_XXXXXX.wav`) to prevent any discrepancies during training.

In [16]:
import soundfile as sf
import io

print("\n" + "=" * 50)
print("SAUVEGARDE DES FICHIERS AUDIO")
print("=" * 50)

saved_files_count = 0
for idx, row in df_combined.iterrows():
    audio_filename = f"audio_{idx:06d}.wav"
    audio_path = os.path.join(wavs_dir, audio_filename)

    # The 'audio' column contains dictionaries with a 'bytes' key
    audio_bytes = row['audio']['bytes']

    # Use BytesIO to treat the bytes as a file
    try:
        # Read audio data from bytes using soundfile
        # soundfile.read returns (data, samplerate)
        data, samplerate = sf.read(io.BytesIO(audio_bytes))

        # Save the audio data to a WAV file
        sf.write(audio_path, data, samplerate)
        saved_files_count += 1
    except Exception as e:
        print(f"❌ Erreur lors de la sauvegarde de {audio_filename}: {e}")

print(f"✅ {saved_files_count} fichiers audio sauvegardés dans {wavs_dir}")

print("\n" + "=" * 50)
print("VÉRIFICATION DE LA QUALITÉ")
print("=" * 50)

# Vérifier les durées audio
durations = []
for idx, row in df_combined.iterrows():
    audio_filename = f"audio_{idx:06d}.wav"
    audio_path = os.path.join(wavs_dir, audio_filename)

    # Skip if file was not successfully saved
    if not os.path.exists(audio_path):
        continue

    try:
        info = sf.info(audio_path)
        durations.append(info.duration)
    except Exception as e:
        print(f"❌ Erreur lors de la lecture de la durée de {audio_filename}: {e}")

import numpy as np
durations = np.array(durations)

print(f"\n📊 Statistiques des durées audio :")
print(f"  • Durée moyenne : {durations.mean():.2f}s")
print(f"  • Durée min : {durations.min():.2f}s")
print(f"  • Durée max : {durations.max():.2f}s")
print(f"  • Durée totale : {durations.sum()/3600:.2f}h")

# Recommandations
if durations.sum() < 3600:
    print("\n⚠️  Attention : moins d'1h de données (recommandé: 5-10h minimum)")
elif durations.sum() < 18000:
    print("\n✓ Quantité acceptable (1-5h)")
else:
    print("\n✅ Bonne quantité de données (>5h)")


SAUVEGARDE DES FICHIERS AUDIO
✅ 75399 fichiers audio sauvegardés dans tts_dataset/wavs

VÉRIFICATION DE LA QUALITÉ

📊 Statistiques des durées audio :
  • Durée moyenne : 2.18s
  • Durée min : 1.02s
  • Durée max : 30.00s
  • Durée totale : 45.71h

✅ Bonne quantité de données (>5h)


In [17]:
import os
import glob
import io
import pandas as pd
import soundfile as sf
import re # Import regex module for robust cleaning
import string # To get standard punctuation

print("\n" + "=" * 50)
print("CRICATION DU FICHIER METADATA")
print("=" * 50)

# CrICATION du fichier metadata au format Coqui TTS
# Format: filename|text|normalized_text
metadata_lines = []

# Define allowed characters from config (as used in config_bambara.json)
# These strings explicitly include a space at the end.
allowed_chars_from_config = "abcdefghijklmnopqrstuvwxyz@áâãäåèéêëìíîïòóôõöùúûüýÿñɛɔŋ "
allowed_punctuation_from_config = "!'(),-.:;? "

# Combine all characters that should be explicitly allowed for cleaning.
# This set includes: lowercase and uppercase English letters, digits, custom Bambara characters,
# specified punctuations, and space.
all_allowed_literal_chars = allowed_chars_from_config + allowed_punctuation_from_config + string.ascii_uppercase + string.digits

# Create a set to remove any duplicate characters and ensure uniqueness.
all_allowed_set = set(all_allowed_literal_chars)
all_allowed_for_regex_str = "".join(sorted(list(all_allowed_set))) # Sort for consistent escaping

# Escape all characters in the combined string for safe use within a regex character set.
escaped_all_allowed = re.escape(all_allowed_for_regex_str)

# The pattern now matches any character NOT in our explicitly defined allowed set.
# This robustly filters out problematic characters (like '|' or unhandled special chars)
# and explicitly includes space, avoiding the SyntaxWarning with '\s'.
chars_to_replace_pattern = re.compile(f'[^' + escaped_all_allowed + ']')

for idx, row in df_combined.iterrows():
    # !!! CORRECTED: Removed '.wav' from the filename here. The formatter will add it. !!!
    audio_filename_col = f"audio_{idx:06d}"
    text_raw = str(row['text'])

    # Step 1: Replace any characters not in our allowed set with a space.
    # This is more robust as it filters out unexpected characters, including problematic pipes.
    text_cleaned_for_col = chars_to_replace_pattern.sub(' ', text_raw)

    # Step 2: Normalize whitespace (multiple spaces to single space, strip leading/trailing)
    text_cleaned_for_col = ' '.join(text_cleaned_for_col.split()).strip()

    # If the cleaned text is empty, skip this entry to avoid malformed lines
    if not text_cleaned_for_col:
        print(f"⚠ï¸ Skipping audio_{idx:06d}.wav due to empty cleaned text after aggressive cleaning.")
        continue

    # LJSpeech format: filename|text|normalized_text
    # We use the same cleaned text for both for simplicity, but ensures structure.
    metadata_lines.append(f"{audio_filename_col}|{text_cleaned_for_col}|{text_cleaned_for_col}")

# Sauvegarder metadata.csv
metadata_path = os.path.join(output_dir, "metadata.csv")
with open(metadata_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(metadata_lines))

print(f"✅ Fichier metadata créé : {metadata_path}")
print(f"📊 Nombre d'entrées : {len(metadata_lines)}")

# Afficher quelques exemples
print("\n📋 Exemples de metadata :")
for line in metadata_lines[:3]:
    print(f"  {line}")


CRICATION DU FICHIER METADATA
✅ Fichier metadata créé : tts_dataset/metadata.csv
📊 Nombre d'entrées : 75399

📋 Exemples de metadata :
  audio_000000|Mankan o man , mankan o mankan , wa se bi mun na dɔɔnin o b'o yɛrɛ ta.|Mankan o man , mankan o mankan , wa se bi mun na dɔɔnin o b'o yɛrɛ ta.
  audio_000001|Ka denmisɛnnin ko ka bɔ bilakoroya nɔgɔ la ?|Ka denmisɛnnin ko ka bɔ bilakoroya nɔgɔ la ?
  audio_000002|Sabula baaraw fɛnɛ ka can an bolo.|Sabula baaraw fɛnɛ ka can an bolo.


## **Divide Data into Train, Validation, and Test Sets**

This step divides the `metadata.csv` file into three separate files: `metadata_train.csv`, `metadata_val.csv`, and `metadata_test.csv`. These files are essential for training and evaluating the Coqui TTS model.

In [18]:
from sklearn.model_selection import train_test_split

print("\n" + "=" * 50)
print("DIVISION DES DONNÉES")
print("=" * 50)

# Lire le metadata
with open(metadata_path, 'r', encoding='utf-8') as f:
    # Filter out empty lines or lines with only whitespace
    all_lines = [line for line in f.readlines() if line.strip()]

# Division 80% train, 10% val, 10% test
train_lines, temp_lines = train_test_split(all_lines, test_size=0.2, random_state=42)
val_lines, test_lines = train_test_split(temp_lines, test_size=0.5, random_state=42)

# Sauvegarder les fichiers
with open(os.path.join(output_dir, "metadata_train.csv"), 'w', encoding='utf-8') as f:
    f.writelines(train_lines)

with open(os.path.join(output_dir, "metadata_val.csv"), 'w', encoding='utf-8') as f:
    f.writelines(val_lines)

with open(os.path.join(output_dir, "metadata_test.csv"), 'w', encoding='utf-8') as f:
    f.writelines(test_lines)

print(f"✅ Train : {len(train_lines)} échantillons")
print(f"✅ Val   : {len(val_lines)} échantillons")
print(f"✅ Test  : {len(test_lines)} échantillons")


DIVISION DES DONNÉES
✅ Train : 60319 échantillons
✅ Val   : 7540 échantillons
✅ Test  : 7540 échantillons


In [28]:
!pip install torchcodec

# **Optimized configuration**

In [35]:
config = {
    "model": "tacotron2",
    "run_name": "bambara_tts_24h",
    "run_description": "Modèle TTS Bambara (Mali) - Optimisé 24 heures - Sauvegarde Drive",

    # ✅ CHEMINS VERS GOOGLE DRIVE
    "output_path": "/content/drive/MyDrive/bambara_tts_output",  # ✅ Sauvegarde directe sur Drive

    "datasets": [{
        "name": "bambara",
        "path": "tts_dataset/",
        "meta_file_train": "metadata_train.csv",
        "meta_file_val": "metadata_val.csv",
        "formatter": "ljspeech"
    }],

    # Paramètres audio
    "audio": {
        "sample_rate": 22050,
        "hop_length": 256,
        "win_length": 1024,
        "fft_size": 1024,
        "num_mels": 80,
        "mel_fmin": 0,
        "mel_fmax": 8000,
        "do_trim_silence": True,
        "trim_db": 45,
        "signal_norm": True,
        "symmetric_norm": False,
        "max_norm": 1.0,
        "clip_norm": True
    },

    # ⚡ PARAMÈTRES OPTIMISÉS POUR 24H
    "batch_size": 64,
    "eval_batch_size": 32,
    "num_loader_workers": 10,
    "num_eval_loader_workers": 5,

    "mixed_precision": True,
    "precision": "fp16",
    "cudnn_benchmark": True,
    "cudnn_deterministic": False,
    "cudnn_enabled": True,

    "grad_accum_steps": 2,

    "run_eval": True,
    "test_delay_epochs": 5,
    "run_eval_steps": 3000,

    "epochs": 100,

    "text_cleaner": "basic_cleaners",
    "use_phonemes": False,
    "phoneme_language": "bm",
    "phoneme_cache_path": f"/content/drive/MyDrive/bambara_tts_output/phoneme_cache",  # ✅ Cache sur Drive
    "enable_eos_bos_chars": True,
    "add_blank": True,

    "print_step": 200,
    "print_eval": False,
    "save_step": 5000,  # ✅ Sauvegarde tous les 5000 steps
    "checkpoint": True,
    "keep_all_best": False,
    "keep_after": 30000,
    "save_best_after": 20000,

    # ⚡ OPTIMISEUR
    "optimizer": "AdamW",
    "lr": 0.0015,
    "lr_scheduler": "NoamLR",
    "lr_scheduler_params": {
        "warmup_steps": 2000
    },
    "scheduler_after_epoch": True,
    "grad_clip": 5.0,
    "weight_decay": 1e-6,
    "betas": [0.9, 0.98],
    "eps": 1e-9,

    # 📝 CARACTÈRES BAMBARA
    "characters": {
        "characters_class": "TTS.tts.utils.text.characters.Graphemes",
        "characters": "aàáâbcdeèéêɛɛ̀ɛ́ɛ̂fghiìíîjklmnoòóôɔɔ̀ɔ́ɔ̂nɲŋprstuùúûwyz",
        "punctuations": "!'(),-.:;? ",
        "phonemes": "abeɛijoɔubpdtgkmnɲŋfshrlwyj ",
        "pad": "_",
        "eos": "~",
        "bos": "^",
        "blank": "<BLNK>"
    },

    # ⚡ TACOTRON2
    "r": 2,
    "attention_type": "original",
    "attention_win": True,
    "windowing": True,
    "use_forward_attn": True,
    "forward_attn_mask": False,
    "location_attn": True,
    "attention_norm": "softmax",
    "prenet_type": "original",
    "prenet_dropout": True,
    "separate_stopnet": True,
    "bidirectional_decoder": False,
    "double_decoder_consistency": False,

    # 🎯 AUGMENTATION
    "use_noise_augment": True,
    "use_pitch_augment": False,
    "use_phoneme_cache": True,

    # 🛑 EARLY STOPPING
    "early_stop": {
        "enabled": True,
        "patience": 25,
        "min_delta": 0.0003
    },

    # 🧪 PHRASES DE TEST
    "test_sentences": [
        "paseke baara hokumu kɔnɔna la",
        "i ka kɛnɛya",
        "aw ni tile",
        "n bɛ bamanankan fɔ",
        "a ka di kosɔbɛ",
        "aw ni sɔgɔma",
        "i tɔgɔ bi di"
    ]
}

# ─────────────────────────────────────────────────────────
# 3. SAUVEGARDER LA CONFIGURATION
# ─────────────────────────────────────────────────────────

# Sauvegarder localement
config_path_local = "config_bambara_12h.json"
with open(config_path_local, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

# Sauvegarder sur Drive
config_path_drive = f"/content/drive/MyDrive/bambara_tts_output/config_bambara_12h.json"
with open(config_path_drive, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print(f"\n✅ Configuration sauvegardée :")
print(f"   - Local : {config_path_local}")
print(f"   - Drive : {config_path_drive}")

# ─────────────────────────────────────────────────────────
# 4. CRÉER UN FICHIER README SUR DRIVE
# ─────────────────────────────────────────────────────────

readme_content = f"""# 🇲🇱 Modèle TTS Bambara - Entraînement 24h

## 📊 Informations

- **Modèle** : Tacotron2
- **Langue** : Bambara (Mali)
- **Durée d'entraînement** : 24 heures (A100) / 30 heures (V100)
- **Dataset** : 60 319 échantillons
- **Qualité attendue** : 8-8.5/10

## 📁 Structure des dossiers
"""


✅ Configuration sauvegardée :
   - Local : config_bambara_12h.json
   - Drive : /content/drive/MyDrive/bambara_tts_output/config_bambara_12h.json


In [ ]:
# Script d'entraînement : train_tts.py
from TTS.bin.train_tts import main
import sys
import os

# Lancer l'entraînement
print("\n" + "=" * 50)
print("DÉMARRAGE DE L'ENTRAÎNEMENT")
print("=" * 50)

# Via Python en modifiant sys.argv
sys.argv = ['train_tts.py', '--config_path', 'config_bambara_12h.json']
main()


DÉMARRAGE DE L'ENTRAÎNEMENT
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset
Found 60318 files in /content/tts_dataset


INFO:TTS.tts.datasets:Found 60318 files in /content/tts_dataset


Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2
Using model: tacotron2


INFO:TTS.tts.models:Using model: tacotron2


Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...
Setting up Audio Processor...


...

INFO:TTS.tts.datasets.dataset:Num. instances discarded samples: 0


Batch group size: 0
Batch group size: 0
Batch group size: 0
Batch group size: 0
Batch group size: 0
Batch group size: 0
Batch group size: 0
Batch group size: 0
Batch group size: 0


INFO:TTS.tts.datasets.dataset:Batch group size: 0

 > TRAINING (2026-01-18 09:28:57) 



   --> TIME: 2026-01-18 09:29:15 -- STEP: 0/942 -- GLOBAL_STEP: 0
     | > current_lr: 7.5e-07  (7.5e-07)
     | > decoder_loss: 0.1402866393327713  (0.1402866393327713)
     | > postnet_loss: 2.1462771892547607  (2.1462771892547607)
     | > stopnet_loss: 0.7083955407142639  (0.7083955407142639)
     | > ga_loss: 0.008961169049143791  (0.008961169049143791)
     | > decoder_diff_spec_loss: 0.04267537593841553  (0.04267537593841553)
     | > postnet_diff_spec_loss: 4.0836100578308105  (4.0836100578308105)
     | > decoder_ssim_loss: 0.9668660163879395  (0.9668660163879395)
     | > postnet_ssim_loss: 0.9452007412910461  (0.9452007412910461)
     | > loss: 2.834430456161499  (2.834430456161499)
     | > align_error: 0.9450247697532177  (0.9450247697532177)
     | > amp_scaler: 65536.0  (65536.0)
     | > grad_norm: tensor(4.3916, device='cuda:0')  (tensor(4.3916, device='cuda:0'))
     | > step_time: 8.9726  (8.972642183303833)
     | > loader_time: 9.0365  (9.036456823348999)




   --> TIME: 2026-01-18 09:31:50 -- STEP: 200/942 -- GLOBAL_STEP: 200
     | > current_lr: 7.5e-07  (7.500000000000002e-07)
     | > decoder_loss: 0.12289757281541824  (0.13710151128470904)
     | > postnet_loss: 2.1144399642944336  (2.1345677030086536)
     | > stopnet_loss: 0.7042900919914246  (0.7048683223128319)
     | > ga_loss: 0.004728763829916716  (0.006617766949348152)
     | > decoder_diff_spec_loss: 0.035836540162563324  (0.03807763729244471)
     | > postnet_diff_spec_loss: 3.9889140129089355  (4.022806029319764)
     | > decoder_ssim_loss: 0.9595158100128174  (0.963304978609085)
     | > postnet_ssim_loss: 0.9473416805267334  (0.9431472402811051)
     | > loss: 2.7701704502105713  (2.7977084386348734)
     | > align_error: 0.9335437566041946  (0.9395113649405539)
     | > amp_scaler: 65536.0  (65536.0)
     | > grad_norm: tensor(3.5270, device='cuda:0')  (tensor(3.3998, device='cuda:0'))
     | > step_time: 0.7616  (0.7577231466770172)
     | > loader_time: 0.0032  (0.005

# Task
Update the configuration in cell `RElGSJdNRw7c` to set `"use_phonemes": False` and `"text_cleaner": "basic_cleaners"` and `"output_path": "tts_train_dir_local"`, then re-execute it. Subsequently, restart the training in cell `ATXNVdjlSf1W` to use grapheme-based input, acknowledging that a custom phonemizer would be necessary for high-quality, phoneme-based Bambara TTS.

## Modify Configuration to Use Graphemes

### Subtask:
Update the configuration to save training outputs locally and confirm grapheme-based text processing.


**Reasoning**:
The user wants to update the configuration to save training outputs locally and confirm grapheme-based text processing. I will modify the 'output_path' to 'tts_train_dir_local', confirm 'use_phonemes' is False, and 'text_cleaner' is 'basic_cleaners' in the provided code block for cell 'RElGSJdNRw7c'.



**Reasoning**:
The previous code failed because the `json` module was not imported. I need to add `import json` at the beginning of the cell to resolve the `NameError`. I will also ensure the `output_path` is set to `"tts_train_dir_local"`, `use_phonemes` to `False`, and `text_cleaner` to `"basic_cleaners"` as per the instructions.



In [4]:
import json
import os

config = {
    "model": "tacotron2",
    "run_name": "bambara_tts_24h",
    "run_description": "Modèle TTS Bambara (Mali) - Optimisé 24 heures - Sauvegarde Locale",

    # ✅ CHEMINS VERS GOOGLE DRIVE
    "output_path": "tts_train_dir_local",  # Changed to local path

    "datasets": [{
        "name": "bambara",
        "path": "tts_dataset/",
        "meta_file_train": "metadata_train.csv",
        "meta_file_val": "metadata_val.csv",
        "formatter": "ljspeech"
    }],

    # Paramètres audio
    "audio": {
        "sample_rate": 22050,
        "hop_length": 256,
        "win_length": 1024,
        "fft_size": 1024,
        "num_mels": 80,
        "mel_fmin": 0,
        "mel_fmax": 8000,
        "do_trim_silence": True,
        "trim_db": 45,
        "signal_norm": True,
        "symmetric_norm": False,
        "max_norm": 1.0,
        "clip_norm": True
    },

    # ⚡ PARAMÈTRES OPTIMISÉS POUR 24H
    "batch_size": 64,
    "eval_batch_size": 32,
    "num_loader_workers": 10,
    "num_eval_loader_workers": 5,

    "mixed_precision": True,
    "precision": "fp16",
    "cudnn_benchmark": True,
    "cudnn_deterministic": False,
    "cudnn_enabled": True,

    "grad_accum_steps": 2,

    "run_eval": True,
    "test_delay_epochs": 5,
    "run_eval_steps": 3000,

    "epochs": 100,

    "text_cleaner": "basic_cleaners",
    "use_phonemes": False,
    "phoneme_language": "bm",
    "phoneme_cache_path": f"/content/drive/MyDrive/bambara_tts_output/phoneme_cache",  # Cache on Drive
    "enable_eos_bos_chars": True,
    "add_blank": True,

    "print_step": 200,
    "print_eval": False,
    "save_step": 5000,  # ✅ Save every 5000 steps
    "checkpoint": True,
    "keep_all_best": False,
    "keep_after": 30000,
    "save_best_after": 20000,

    # ⚡ OPTIMISEUR
    "optimizer": "AdamW",
    "lr": 0.0015,
    "lr_scheduler": "NoamLR",
    "lr_scheduler_params": {
        "warmup_steps": 2000
    },
    "scheduler_after_epoch": True,
    "grad_clip": 5.0,
    "weight_decay": 1e-6,
    "betas": [0.9, 0.98],
    "eps": 1e-9,

    # 📝 CARACTÈRES BAMBARA
    "characters": {
        "characters_class": "TTS.tts.utils.text.characters.Graphemes",
        "characters": "aàáâbcdeèéêɛɛ̀ɛ́ɛ̂fghiìíîjklmnoòóôɔɔ̀ɔ́ɔ̂nɲŋprstuùúûwyz",
        "punctuations": "!'(),-.:;? ",
        "phonemes": "abeɛijoɔubpdtgkmnɲŋfshrlwyj ",
        "pad": "_",
        "eos": "~",
        "bos": "^",
        "blank": "<BLNK>"
    },

    # ⚡ TACOTRON2
    "r": 2,
    "attention_type": "original",
    "attention_win": True,
    "windowing": True,
    "use_forward_attn": True,
    "forward_attn_mask": False,
    "location_attn": True,
    "attention_norm": "softmax",
    "prenet_type": "original",
    "prenet_dropout": True,
    "separate_stopnet": True,
    "bidirectional_decoder": False,
    "double_decoder_consistency": False,

    # 🎯 AUGMENTATION
    "use_noise_augment": True,
    "use_pitch_augment": False,
    "use_phoneme_cache": True,

    # 🛑 EARLY STOPPING
    "early_stop": {
        "enabled": True,
        "patience": 25,
        "min_delta": 0.0003
    },

    # 🧪 PHRASES DE TEST
    "test_sentences": [
        "paseke baara hokumu kɔnɔna la",
        "i ka kɛnɛya",
        "aw ni tile",
        "n bɛ bamanankan fɔ",
        "a ka di kosɔbɛ",
        "aw ni sɔgɔma",
        "i tɔgɔ bi di"
    ]
}

# ─────────────────────────────────────────────────────────
# 3. SAUVEGARDER LA CONFIGURATION
# ─────────────────────────────────────────────────────────

# Sauvegarder localement
config_path_local = "config_bambara_12h.json"
with open(config_path_local, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

# Sauvegarder sur Drive (for consistency, though output will be local)
config_path_drive = f"/content/drive/MyDrive/bambara_tts_output/config_bambara_12h.json"
with open(config_path_drive, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print(f"\n✅ Configuration sauvegardée :")
print(f"   - Local : {config_path_local}")
print(f"   - Drive : {config_path_drive}")

# ─────────────────────────────────────────────────────────
# 4. CRÉER UN FICHIER README SUR DRIVE
# ─────────────────────────────────────────────────────────

readme_content = f"""# 🇲🇱 Modèle TTS Bambara - Entraînement 24h\n\n## 📊 Informations\n\n- **Modèle** : Tacotron2\n- **Langue** : Bambara (Mali)\n- **Durée d'entraînement** : 24 heures (A100) / 30 heures (V100)\n- **Dataset** : 60 319 échantillons\n- **Qualité attendue** : 8-8.5/10\n\n## 📁 Structure des dossiers\n"""


✅ Configuration sauvegardée :
   - Local : config_bambara_12h.json
   - Drive : /content/drive/MyDrive/bambara_tts_output/config_bambara_12h.json


## Start Training with Local Output and Explain Drive Sync

Now that the `output_path` in the configuration is set to `tts_train_dir_local`, all model checkpoints, logs, and generated samples will be saved to this local folder within the Colab environment during training. This ensures continuous saving without constant interaction with Google Drive, which can sometimes be slower or lead to issues during frequent writes.

### Manual Syncing Strategy for Google Drive

Since the training process is a blocking operation, direct, continuous synchronization to Google Drive after every epoch or step is not automatically configurable within the Coqui TTS training loop itself. To ensure your progress is saved persistently on Google Drive, you have two primary options:

1.  **Periodic Manual Copies**: During long training runs, you can manually stop the training (e.g., using the stop button in Colab or by interrupting the kernel) at a convenient point (e.g., after several epochs or when a significant checkpoint is saved). Then, you would execute a separate code cell to copy the `tts_train_dir_local` folder to your designated Google Drive path. After the copy is complete, you can resume training from the last saved checkpoint.

2.  **Post-Training/Interruption Copy**: The simplest method is to let the training run its course until it either completes all epochs, hits an early stopping condition, or you manually decide to stop it. Once the training process has ceased, a single command can be used to copy the entire `tts_train_dir_local` directory to your Google Drive. This ensures all your work, including the final model and logs, is backed up.

We will now proceed to start the training. After the training is manually stopped or completed, I will provide a code block to copy the entire local output to Google Drive.


**Reasoning**:
Now that the configuration has been updated to use local output, I will execute the training cell (`ATXNVdjlSf1W`) to start the Coqui TTS training process, which will save all outputs to `tts_train_dir_local`.



# Task
Start the Coqui TTS training process with the output path set to `tts_train_dir_local`.

## Start Training with Local Output Path

### Subtask:
Execute the training process with the updated configuration, saving all outputs to the local directory `tts_train_dir_local`.


**Reasoning**:
The subtask is to execute the training process with the updated configuration, which means running the `train_tts.py` script using the previously updated configuration file. Executing cell `ATXNVdjlSf1W` will achieve this, as it calls the `main()` function with the specified configuration.



In [ ]:
# Script d'entraînement : train_tts.py
from TTS.bin.train_tts import main
import sys
import os

# Lancer l'entraînement
print("\n" + "=" * 50)
print("DÉMARRAGE DE L'ENTRAÎNEMENT")
print("=" * 50)

# Via Python en modifiant sys.argv
sys.argv = ['train_tts.py', '--config_path', 'config_bambara_12h.json']
main()


DÉMARRAGE DE L'ENTRAÎNEMENT
Found 60318 files in /content/tts_dataset


INFO:TTS.tts.datasets:Found 60318 files in /content/tts_dataset


Using model: tacotron2


INFO:TTS.tts.models:Using model: tacotron2


Setting up Audio Processor...


...

INFO:TTS.tts.datasets.dataset:Num. instances discarded samples: 0


Batch group size: 0


INFO:TTS.tts.datasets.dataset:Batch group size: 0

 > TRAINING (2026-01-18 09:53:17) 



   --> TIME: 2026-01-18 09:53:30 -- STEP: 0/942 -- GLOBAL_STEP: 0
     | > current_lr: 7.5e-07  (7.5e-07)
     | > decoder_loss: 0.14692817628383636  (0.14692817628383636)
     | > postnet_loss: 2.1561312675476074  (2.1561312675476074)
     | > stopnet_loss: 0.7089093327522278  (0.7089093327522278)
     | > ga_loss: 0.008302861824631691  (0.008302861824631691)
     | > decoder_diff_spec_loss: 0.046952247619628906  (0.046952247619628906)
     | > postnet_diff_spec_loss: 4.106233596801758  (4.106233596801758)
     | > decoder_ssim_loss: 0.9663951992988586  (0.9663951992988586)
     | > postnet_ssim_loss: 0.945092499256134  (0.945092499256134)
     | > loss: 2.8423566818237305  (2.8423566818237305)
     | > align_error: 0.9411415234208107  (0.9411415234208107)
     | > amp_scaler: 65536.0  (65536.0)
     | > grad_norm: tensor(3.8617, device='cuda:0')  (tensor(3.8617, device='cuda:0'))
     | > step_time: 7.65  (7.650017976760864)
     | > loader_time: 6.0875  (6.0875160694122314)




   --> TIME: 2026-01-18 09:56:04 -- STEP: 200/942 -- GLOBAL_STEP: 200
     | > current_lr: 7.5e-07  (7.500000000000002e-07)
     | > decoder_loss: 0.12875360250473022  (0.14325598601251832)
     | > postnet_loss: 2.1170973777770996  (2.1413742899894714)
     | > stopnet_loss: 0.7134299874305725  (0.7090991833806037)
     | > ga_loss: 0.006489890627563  (0.006750146802514792)
     | > decoder_diff_spec_loss: 0.03855161368846893  (0.041649039201438406)
     | > postnet_diff_spec_loss: 3.9861903190612793  (4.038934189081194)
     | > decoder_ssim_loss: 0.9632460474967957  (0.9616983938217163)
     | > postnet_ssim_loss: 0.9470607042312622  (0.9431999301910401)
     | > loss: 2.791104555130005  (2.810377860069275)
     | > align_error: 0.9396383985877037  (0.9374929769337177)
     | > amp_scaler: 65536.0  (65536.0)
     | > grad_norm: tensor(3.3678, device='cuda:0')  (tensor(3.8581, device='cuda:0'))
     | > step_time: 0.9348  (0.7548631083965296)
     | > loader_time: 0.0064  (0.0059137

**Reasoning**:
The previous training run was interrupted. To restart the training process as requested, I will re-execute the training cell.



In [ ]:
# Script d'entraînement : train_tts.py
from TTS.bin.train_tts import main
import sys
import os

# Lancer l'entraînement
print("\n" + "=" * 50)
print("DÉMARRAGE DE L'ENTRAÎNEMENT")
print("=" * 50)

# Via Python en modifiant sys.argv
sys.argv = ['train_tts.py', '--config_path', 'config_bambara_12h.json']
main()